# Construction du classifieur


In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import random
import pandas as pd
import cv2
import os
from skimage import io, util
from skimage.transform import resize


# Ah stylé d'avoir trouvé ça !
from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    average_precision_score,
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

## Récupération des jeux


## Découper les écocups et les négatifs


On découpe les positifs en fonction de bbox


On va découper des bouts d'images négatives


## Transformation


Supprimer les images vides


In [10]:
# Chargement des données
pos_patch_fs = [f for f in  os.listdir(os.path.join("local_data", "4_normalized_patches", "pos")) if f.endswith(".jpg")]
neg_patch_fs = [f for f in  os.listdir(os.path.join("local_data", "4_normalized_patches", "neg")) if f.endswith(".jpg")]

img_pos_train_cut_resized = []
img_neg_train_cut_resized = []

for f in pos_patch_fs:
    try:
        patch = plt.imread(
            os.path.join("local_data", "4_normalized_patches", "pos", f)
        )
        img_pos_train_cut_resized.append(patch[:,:,0]) # le channel grayscale est dupliqué sur les 3 chanaux RGB (on en isole 1)

    except FileNotFoundError:
        continue

for f in neg_patch_fs:
    try:
        patch = plt.imread(
            os.path.join("local_data", "4_normalized_patches", "neg", f)
        )
        img_neg_train_cut_resized.append(patch[:,:,0]) # le channel grayscale est dupliqué sur les 3 chanaux RGB (on en isole 1)

    except FileNotFoundError:
        continue

In [ ]:
print(len(img_pos_train_cut_resized))
print(len(img_neg_train_cut_resized))

print(img_pos_train_cut_resized[0].shape) # on est bien en grayscale

1204
6020
(128, 72)


La commande ci-dessous permet de convertir
les valeurs des pixels à valeurs entières [[0; 255]] en valeurs flottantes [0; 1]


In [7]:
img_pos_train_cut_resized_float = [
    util.img_as_float(img) for img in img_pos_train_cut_resized
]
img_neg_train_cut_resized_float = [
    util.img_as_float(img) for img in img_neg_train_cut_resized
]

On souhaite utiliser directement les pixels comme représentation des images. La commande
I.flatten() permet d’aplatir l’image pour obtenir un vecteur

TODO : essayer d'autres extracteurs de features (cf. OneNote)


In [8]:
img_pos_train_cut_resized_float_flatten = [
    img.flatten() for img in img_pos_train_cut_resized_float
]
img_neg_train_cut_resized_float_flatten = [
    img.flatten() for img in img_neg_train_cut_resized_float
]

## Construire les jeux


In [9]:
# les cas difficiles ne sont pas inclus -> décision dans "splitter.py"
# (Et comme je t'ai montré, les cas difficiles sont vraiment dégueulasse, j'ai clean manuellement que les cas faciles)
# (Ca sera pas un problème avec l'algo de détection itératif) (cf cours 10)

y = []
X = img_pos_train_cut_resized_float_flatten + img_neg_train_cut_resized_float_flatten
for _ in range(len(img_pos_train_cut_resized_float_flatten)):
    y.append(1)

for _ in range(len(img_neg_train_cut_resized_float_flatten)):
    y.append(0)

y_clean = np.array(y)

#### Créer une pondération pour les exemples difficiles


In [10]:
print(len(X), len(y))

7296 7296


#### Diviser le jeu

TODO : peut-être une validation croisée (déjà implémentée par des bibliothèques, cf. TD6)


In [11]:
train_part = 80 / 100
train_size = int(len(X) * train_part)

# Problème: tout est dans l'ordre, nous devons donc associer chaque image à sa vérité et mélanger les données
indices = np.arange(len(X))
np.random.shuffle(indices)
X_shuffled = np.array(X)[indices]
y_shuffled = np.array(y_clean)[indices]


X_train = X_shuffled[0:train_size]
y_train = y_shuffled[0:train_size]
X_validation = X_shuffled[train_size:]
y_validation = y_shuffled[train_size:]

print(len(X_train))
print(len(y_train))
print(len(X_validation))
print(len(y_validation))

5836
5836
1460
1460


## Choix du classifieur


Nous allons choisir le classifieur le plus efficace parmis un certain nombre de classifieurs


In [14]:
def test_model(
    name, model, X_train, y_train, X_validation, y_validation, df_resultat_clf
):

    print(f"Testing {name}...")

    # model.fit(X_train, y_train, sample_weight=weight)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_validation)

    accuracy = accuracy_score(y_validation, y_pred)

    error = (1 - accuracy) * 100
    # Calculer le rappel et la précision
    rappel = recall_score(y_validation, y_pred)
    precision = precision_score(y_validation, y_pred)
    f1_score = 2 * (precision * rappel) / (precision + rappel)
    avg_precision_score = average_precision_score(y_validation, y_pred)

    df_resultat_clf = pd.concat(
        [
            df_resultat_clf,
            pd.DataFrame(
                [
                    {
                        "model": name,
                        "accuracy": accuracy,
                        "error%": error,
                        "rappel": rappel,
                        "precision": precision,
                        "f1_score": f1_score,
                        "average_precision_score": avg_precision_score,
                    }
                ]
            ),
        ],
        ignore_index=True,
    )
    return df_resultat_clf

In [13]:
def get_models_dict():

    models_dict = {}
    # KNN
    neighbors_list = [3, 25, 100, 250]
    for neighbors in neighbors_list:
        knn = KNeighborsClassifier(n_neighbors=neighbors)
        models_dict[f"KNN (k={neighbors})"] = knn

    # Decision Tree
    decision_tree = DecisionTreeClassifier()
    models_dict["Decision Tree"] = decision_tree

    # Random Forest
    estimators_list = [3, 25, 100, 250]
    for estimators in estimators_list:
        random_forest = RandomForestClassifier(n_estimators=estimators)
        models_dict[f"Random Forest (n_estimators={estimators})"] = random_forest

    # SVC
    svc = SVC()
    models_dict["SVC"] = svc

    # LinearSVC
    linear_svc = LinearSVC()
    models_dict["Linear SVC"] = linear_svc

    # SVC kernel
    svc_kernel_list = ["poly", "rbf", "sigmoid"]
    for kernel in svc_kernel_list:
        svc_kernel = SVC(kernel=kernel)
        models_dict[f"SVC (kernel={kernel})"] = svc_kernel

    # Logistic Regression
    iter_list = [25, 200, 400]
    for n_iter in iter_list:
        logistic_regression = LogisticRegression(max_iter=n_iter)
        models_dict[f"Logistic Regression (max_iter={n_iter})"] = logistic_regression

    # AdaBoost
    estimators_list = [10, 25, 100]
    for estimators in estimators_list:
        ada_boost = AdaBoostClassifier(n_estimators=estimators)
        models_dict[f"AdaBoost (n_estimators={estimators})"] = ada_boost

    return models_dict

    # Gradient Boosting
    learning_rate_list = [0.01, 0.2, 0.5]
    for learning_rate in learning_rate_list:
        gradient_boosting = GradientBoostingClassifier(learning_rate=learning_rate)
        models_dict[f"Gradient Boosting (learning_rate={learning_rate})"] = (
            gradient_boosting
        )
    return models_dict

In [15]:
df_resultat_clf = pd.DataFrame(
    columns=[
        "model",
        "accuracy",
        "error%",
        "rappel",
        "precision",
        "f1_score",
        "average_precision_score",
    ]
)

models_dict = get_models_dict()


for name, model in models_dict.items():
    df_resultat_clf = test_model(
        name, model, X_train, y_train, X_validation, y_validation, df_resultat_clf
    )

df_resultat_clf = df_resultat_clf.sort_values(by="error%")
print(df_resultat_clf)

Testing KNN (k=3)...


C:\Users\Adrien\AppData\Local\Temp\ipykernel_23452\3502999163.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_resultat_clf = pd.concat(


Testing KNN (k=25)...
Testing KNN (k=100)...
Testing KNN (k=250)...
Testing Decision Tree...
Testing Random Forest (n_estimators=3)...
Testing Random Forest (n_estimators=25)...
Testing Random Forest (n_estimators=100)...
Testing Random Forest (n_estimators=250)...
Testing SVC...
Testing Linear SVC...


c:\Dossier_perso\Projets\Prog\SY32_detection_ecocup\venv\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Dossier_perso\Projets\Prog\SY32_detection_ecocup\venv\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Testing SVC (kernel=poly)...
Testing SVC (kernel=rbf)...
Testing SVC (kernel=sigmoid)...
Testing Logistic Regression (max_iter=25)...


...

Testing AdaBoost (n_estimators=10)...
Testing AdaBoost (n_estimators=25)...


c:\Dossier_perso\Projets\Prog\SY32_detection_ecocup\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Testing AdaBoost (n_estimators=100)...


c:\Dossier_perso\Projets\Prog\SY32_detection_ecocup\venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


                                 model  accuracy     error%    rappel  \
9                                  SVC  0.922603   7.739726  0.547325   
12                    SVC (kernel=rbf)  0.922603   7.739726  0.547325   
11                   SVC (kernel=poly)  0.916438   8.356164  0.654321   
8     Random Forest (n_estimators=250)  0.909589   9.041096  0.460905   
7     Random Forest (n_estimators=100)  0.909589   9.041096  0.469136   
6      Random Forest (n_estimators=25)  0.909589   9.041096  0.473251   
19         AdaBoost (n_estimators=100)  0.902055   9.794521  0.551440   
14   Logistic Regression (max_iter=25)  0.896575  10.342466  0.547325   
0                            KNN (k=3)  0.888356  11.164384  0.386831   
15  Logistic Regression (max_iter=200)  0.884247  11.575342  0.592593   
18          AdaBoost (n_estimators=25)  0.870548  12.945205  0.320988   
16  Logistic Regression (max_iter=400)  0.863014  13.698630  0.572016   
17          AdaBoost (n_estimators=10)  0.863014  1

In [1]:
df_resultat_clf.head()

NameError: name 'df_resultat_clf' is not defined

In [171]:
## Affichons les 3 meilleurs résultats pour chaque score
# accuracy
top_accuracy = df_resultat_clf.nlargest(3, "accuracy")
print("Top 3 Accuracy:")
print(top_accuracy[["model", "accuracy"]])

# rappel
top_recall = df_resultat_clf.nlargest(3, "rappel")
print("\nTop 3 Rappel:")
print(top_recall[["model", "rappel"]])

# précision
top_precision = df_resultat_clf.nlargest(3, "precision")
print("\nTop 3 Précision:")
print(top_precision[["model", "precision"]])

# f1_score
top_f1_score = df_resultat_clf.nlargest(3, "f1_score")
print("\nTop 3 F1 Score:")
print(top_f1_score[["model", "f1_score"]])

# average_precision_score
top_avg_precision = df_resultat_clf.nlargest(3, "average_precision_score")
print("\nTop 3 Average Precision Score:")
print(top_avg_precision[["model", "average_precision_score"]])

Top 3 Accuracy:
                                    model  accuracy
23  Gradient Boosting (learning_rate=0.3)  0.747312
8        Random Forest (n_estimators=250)  0.741935
7        Random Forest (n_estimators=100)  0.741935

Top 3 Rappel:
               model    rappel
3        KNN (k=250)  0.862745
9                SVC  0.843137
12  SVC (kernel=rbf)  0.843137

Top 3 Précision:
                               model  precision
0                          KNN (k=3)   0.816667
11                 SVC (kernel=poly)   0.784946
7   Random Forest (n_estimators=100)   0.764706

Top 3 F1 Score:
                                    model  f1_score
8        Random Forest (n_estimators=250)  0.775701
23  Gradient Boosting (learning_rate=0.3)  0.775120
5          Random Forest (n_estimators=3)  0.766355

Top 3 Average Precision Score:
                                    model  average_precision_score
11                      SVC (kernel=poly)                 0.717689
23  Gradient Boosting (learning_rate

In [172]:
# enregistrer le dataframe
df_resultat_clf.to_csv("resultats_classification.csv", index=False)